In [ ]:
!pip install nibiru==0.5.0a2 --quiet

> ⚠ **Warning** : Once the above command ended, please restart the python kernel using `⌘/Ctrl + M .`

# Introduction to py-sdk

The python sdk allows you to create queries and transaction to a Nibiru chain.

It allows to interact with all of the modules and can be leveraged to automate trading strategies or monitor them.

This notebook will guide you on opening and closing positions.

In [ ]:
import os
import json
from tqdm import tqdm

import nibiru
import nibiru.msg

In [ ]:
# We set up the environment variables to connect to devnet
envs = {
  "HOST" : "34.130.24.87",
  "LCD_ENDPOINT" : "http://34.130.24.87:1317",
  "GRPC_ENDPOINT" : "34.130.24.87:9090",
  "TENDERMINT_RPC_ENDPOINT" : "http://34.130.24.87:26657",
  "WEBSOCKET_ENDPOINT" : "ws://34.130.24.87:26657/websocket",
  "CHAIN_ID" : "nibiru-localnet-0",
}

os.environ.update(envs)

In [ ]:
pair = "ubtc:unusd"

In [ ]:
tx_config = nibiru.TxConfig(tx_type=nibiru.common.TxType.BLOCK)
agent = (
    nibiru.Sdk
    .authorize()
    .with_network(nibiru.Network.customnet(), insecure=True)
    .with_config(tx_config)
)

Our trader don't have any fund, so if we try to open a position it will fail saying that the address does not exist (never registered with the auth module of the chain).

Let's use the validator funds to give some unusd and unibi to the trader

In [ ]:
whale_mnemonic = "guard cream sadness conduct invite crumble clock pudding hole grit liar hotel maid produce squeeze return argue turtle know drive eight casino maze host"

validator = (
    nibiru.Sdk
    .authorize(whale_mnemonic)
    .with_network(nibiru.Network.customnet(), insecure=True)
    .with_config(tx_config)
)

In [ ]:
_ = validator.tx.execute_msgs(
    nibiru.msg.MsgSend(
        from_address=validator.address,
        to_address=agent.address,
        coins=[nibiru.Coin(10000, "unibi"), nibiru.Coin(10000, "unusd")]
    )
)

We can check the new balance of the trader:

In [ ]:
agent.query.get_bank_balances(agent.address)

Now with those fresh token, we can open a long position

In [ ]:
_ = agent.tx.execute_msgs(
    nibiru.msg.MsgOpenPosition(
        sender=agent.address,
        token_pair="ubtc:unusd",
        side=nibiru.common.Side.BUY,
        quote_asset_amount=10,
        leverage=10,
        base_asset_amount_limit=0,
    )
)

In [ ]:
def print_position():
    print(
        json.dumps(
            agent.query.perp.position(trader=agent.address, token_pair="ubtc:unusd"),
            indent=4,
        )
    )

print_position()

We can then add margin to our position

In [ ]:
tx_output = agent.tx.execute_msgs(
    nibiru.msg.MsgAddMargin(
        sender=agent.address,
        token_pair=pair,
        margin=nibiru.Coin(10, "unusd"),
    )
)

In [ ]:
print_position()

We can then remove it

In [ ]:
tx_output = agent.tx.execute_msgs(
    nibiru.msg.MsgRemoveMargin(
        sender=agent.address,
        token_pair=pair,
        margin=nibiru.Coin(5, "unusd"),
    )
)

In [ ]:
print_position()

We can then close totally the position

In [ ]:
tx_output = agent.tx.execute_msgs(
    nibiru.msg.MsgClosePosition(
        sender=agent.address, 
        token_pair=pair
    )
)

## Multi-transactions

We can build multiple messages into a single transactions to be sure that they are executed consecutively.

It can be useful for example to send tokens after removing margins from a position

Lets create a transction with:
- Agent open a position
- Agent remove some margin from the position
- Agent send this margin to a random address
- Agent close the position

In [ ]:
_ = agent.tx.execute_msgs(
    [
        nibiru.msg.MsgOpenPosition(
            sender=agent.address,
            token_pair="ubtc:unusd",
            side=nibiru.common.Side.BUY,
            quote_asset_amount=100,
            base_asset_amount_limit=0,
            leverage=5,
        ),
        nibiru.msg.MsgRemoveMargin(
            sender=agent.address,
            token_pair=pair,
            margin=nibiru.Coin(5, "unusd"),
        ),
        nibiru.msg.MsgSend(
            from_address=agent.address,
            to_address=validator.address,
            coins=[nibiru.Coin(5, "unusd")]            
        ),
        nibiru.msg.MsgClosePosition(
            sender=agent.address, 
            token_pair=pair
        )                
    ]
)

## Arbitrage strategy

Let's look at the current btc:usd mark price in our chain

In [ ]:
reserves = validator.query.vpool.reserve_assets(pair=pair)
reserves

We can get the price by dividing the quote asset reserve by the base asset reserve:

In [ ]:
current_price = reserves["quote_asset_reserve"] / reserves["base_asset_reserve"]
current_price

BTC:USD spot is currently at 19,193.70USD. Which means that we need short positions opened to move the mark price down to the index price

In [ ]:
import math
def quote_needed_to_move_price(current_price, target_price, quote_reserve) -> float:
    """
    Compute how much quote is needed to move the pool current_price to target_price. This does not include fees.
    Demonstration is left as an exercise (involve playing a bit with the xy=k formula)

    Returns:
        float: Amount of quote to swap
    """
    qp = target_price / current_price
    return -(quote_reserve / math.sqrt(qp) - quote_reserve)

In [ ]:
short_needed = quote_needed_to_move_price(20000, 19193.7, reserves["quote_asset_reserve"])
short_needed

A short position of -415764259331 unusd (-415,764 USD) is needed in order to move the mark price to the current index price of BTC. We want to open this in small steps though since we don't want to trade over the current fluctuation limit. We will do that in 10 transactions.

In [ ]:
for _ in tqdm(range(10)):
    validator.tx.execute_msgs(
        nibiru.msg.MsgOpenPosition(
            sender=validator.address, 
            token_pair="ubtc:unusd", 
            side=nibiru.common.Side.SELL, 
            quote_asset_amount=abs(short_needed)/10, 
            leverage=1, 
            base_asset_amount_limit=100000000000
        )
    )

We can now ensure that the new price is almost equal to 19,193

In [ ]:
reserves = validator.query.vpool.reserve_assets(pair=pair)
new_price = reserves["quote_asset_reserve"] / reserves["base_asset_reserve"]
new_price

In [ ]:
validator.tx.execute_msgs(
    nibiru.msg.MsgClosePosition(
        sender=validator.address, 
        token_pair="ubtc:unusd", 
    )
)